In [1]:
import pandas as pd

In [6]:
df = pd.read_csv('../data/risco-credito.csv')
df.head()

,historia,divida,garantias,renda,risco
0,ruim,alta,nenhuma,0_15,alto
1,desconhecida,alta,nenhuma,15_35,alto
2,desconhecida,baixa,nenhuma,15_35,moderado
3,desconhecida,baixa,nenhuma,acima_35,alto
4,desconhecida,baixa,nenhuma,acima_35,baixo


## Separando classe e previsores

In [10]:
previsores = df.iloc[:,:4].values
previsores[:5]

array([['ruim', 'alta', 'nenhuma', '0_15'],
       ['desconhecida', 'alta', 'nenhuma', '15_35'],
       ['desconhecida', 'baixa', 'nenhuma', '15_35'],
       ['desconhecida', 'baixa', 'nenhuma', 'acima_35'],
       ['desconhecida', 'baixa', 'nenhuma', 'acima_35']], dtype=object)

In [12]:
classe = df.iloc[:,4].values
classe[:5]

array(['alto', 'alto', 'moderado', 'alto', 'baixo'], dtype=object)

## Tratando var. categóricas

In [13]:
from sklearn.preprocessing import LabelEncoder

In [18]:
labelencoder = LabelEncoder()

for i in range(4):
    previsores[:, i] = labelencoder.fit_transform(previsores[:, i])
    
previsores[:5]

array([[3, 0, 1, 0],
       [2, 0, 1, 1],
       [2, 1, 1, 1],
       [2, 1, 1, 2],
       [2, 1, 1, 2]], dtype=object)

## Ávore de decisão

In [25]:
from sklearn.tree import DecisionTreeClassifier, export

In [26]:
classificador = DecisionTreeClassifier(criterion='entropy')
classificador.fit(previsores, classe)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [27]:
print('Importância de cada atributo:\n', classificador.feature_importances_)

Importância de cada atributo:
 [0.41391643 0.06408636 0.03885431 0.4831429 ]


In [33]:
# http://newstudyclub.blogspot.com/2017/02/how-to-install-and-use-graphviz-on.html
export.export_graphviz(classificador, 
                       out_file='arvore.dot', 
                       feature_names=['historia', 'divida', 'garantias', 'renda'],
                       class_names=['alto', 'moderado', 'baixo'],
                       filled=True,
                       leaves_parallel=True # folhas no mesmo nível
                      )
# dot -Tpng arvore.dot  -o arvore_risco.png

In [35]:
# historico bom, divida alta, garantia nenhuma, renda > 35
# historico ruim, divida alta, garantida adequada, renda < 15
resultado = classificador.predict([[0,0,1,2], [3,0,0,0]])
resultado

array(['baixo', 'alto'], dtype=object)